In [7]:
import numpy as np
from random import *

In [354]:
def train(state, spielfeld, learned, epsilon, gamma, alpha, rounds):
    not_found = True
#     for i in range(rounds):
    while(not_found):
        n_state = [0,0]
        indices = np.array([])
        possible_actions = np.array([])
        index_current = state[0]*5 + state[1]
        if not state[0] == 4:
            # darf nach unten gehen
            indices = np.append(indices,[index_current + 5])
            possible_actions = np.append(possible_actions,["down"])
        if not state[0] == 0:
            #darf nach oben gehen
            indices = np.append(indices,[index_current - 5])
            possible_actions = np.append(possible_actions,["up"])
        if not state[1] == 0:
            # darf nach links gehen
            indices = np.append(indices,[index_current - 1])
            possible_actions = np.append(possible_actions,["left"])
        if not state[1] == 4:
            # darf nach rechts gehen
            indices = np.append(indices,[index_current + 1])
            possible_actions = np.append(possible_actions,["right"])
        
        random = uniform(0,1)
        if epsilon > random:
            # Random Choice
            choice = np.random.choice(possible_actions)
            print("Random action:", state, choice)
        else:
            # Hole Werte der möglichen Folgezustände
            values = learned[indices.astype(int),:]
            b = values.flatten()
            optima = np.random.choice(np.flatnonzero(b == b.max()))            
            choice = possible_actions[optima // 4]
            best_value = values.flatten()[optima]
            print(state, choice)

        # get Reward
        if(choice == "down"):
            action_index = 0
            n_state[0] = state[0] + 1
            n_state[1] = state[1]
        elif(choice == "up"):
            action_index = 1
            n_state[0] = state[0] - 1
            n_state[1] = state[1]
        elif(choice == "left"):
            action_index = 2
            n_state[0] = state[0]
            n_state[1] = state[1] - 1
        elif(choice == "right"):
            action_index = 3
            n_state[0] = state[0]
            n_state[1] = state[1] + 1
        
        if epsilon > random:
            best_value = np.max(learned[n_state[0] * n_state[1], :])
        
        reward = get_reward(state,spielfeld,choice)
        current_value = learned[index_current,action_index]
        
        learned[index_current, action_index] = current_value + alpha * (reward + gamma * best_value - current_value)

        state = n_state
        if state == [4,4]:
            state = [0,0]
            learned[24, :] = 1 + np.max([learned[23, :], learned[19, :]])
            not_found = False
            print("-----------found!")
    print("-------- Down -------- Up -------- Left -------- Right")
    print(learned)

In [356]:
def get_reward(state,spielfeld,action):
    x,y = 0,0
    if action == "left":
        x = -1
    elif action == "right":
        x = 1
    elif action == "down":
        y = 1
    elif action == "up":
        y = -1
        
    x +=state[1]    
    y +=state[0]
#   1 = Coin, 2 = Enemy
    if spielfeld[y,x] == 1:
        return 10
    elif spielfeld[y,x] == 2:
        return -50
    else:
        return -1

In [357]:
def test(state, spielfeld, learned):
    n_state = [0,0]
    for i in range(24):    
        index_current = state[0]*5 + state[1]
        action_index = np.array([])
        possible_actions = np.array([])

        if not state[0] == 4:
            # darf nach unten gehen
            possible_actions = np.append(possible_actions,["down"])
            action_index = np.append(action_index, 0)
        if not state[0] == 0:
            #darf nach oben gehen
            possible_actions = np.append(possible_actions,["up"])
            action_index = np.append(action_index, 1)
        if not state[1] == 0:
            # darf nach links gehen
            possible_actions = np.append(possible_actions,["left"])
            action_index = np.append(action_index, 2)
        if not state[1] == 4:
            # darf nach rechts gehen
            possible_actions = np.append(possible_actions,["right"])
            action_index = np.append(action_index, 3)

        # Hole maximalen Wert aus learned
        chosen_action_index = np.argmax(learned[index_current, action_index.astype(int)])
        best_choice = possible_actions[chosen_action_index]
        print(state, best_choice)

        # get Reward
        if(best_choice == "down"):
            n_state[0] = state[0] + 1
            n_state[1] = state[1]
        elif(best_choice == "up"):
            n_state[0] = state[0] - 1
            n_state[1] = state[1]
        elif(best_choice == "left"):
            n_state[0] = state[0]
            n_state[1] = state[1] - 1
        elif(best_choice == "right"):
            n_state[0] = state[0]
            n_state[1] = state[1] + 1

        state = n_state
        if state == [4,4]:
            state = [0,0]
            print("-----------found!")

In [404]:
# Coin = 1, Enemy = 2
spielfeld = np.zeros([5,5])
spielfeld[4,4] = 1
# spielfeld[2,0] = 1
spielfeld[1,0:3] = 2
spielfeld[1,4] = 2
spielfeld[3,0:3] = 2
spielfeld[3,4] = 2
spielfeld = spielfeld.astype(int)
print(spielfeld)

state = [0,0]
learned = np.zeros([25,4])
learned[24,:] = 1
learned[24,:] = 1
epsilon = 0.2
gamma = 0.3
alpha = 0.1
rounds = 40

[[0 0 0 0 0]
 [2 2 2 0 2]
 [0 0 0 0 0]
 [2 2 2 0 2]
 [0 0 0 0 1]]


In [480]:
train(state, spielfeld, learned, epsilon, gamma, alpha, rounds)

[0, 0] down
[1, 0] up
[0, 0] right
Random action: [0, 1] right
[0, 2] left
Random action: [0, 1] down
[1, 1] up
[0, 1] right
[0, 2] left
[0, 1] left
[0, 0] down
[1, 0] up
[0, 0] down
[1, 0] up
[0, 0] right
[0, 1] right
[0, 2] left
[0, 1] left
[0, 0] down
Random action: [1, 0] up
[0, 0] down
[1, 0] up
[0, 0] down
[1, 0] up
[0, 0] down
Random action: [1, 0] right
[1, 1] up
[0, 1] left
[0, 0] right
[0, 1] right
Random action: [0, 2] right
[0, 3] left
Random action: [0, 2] right
[0, 3] left
[0, 2] left
[0, 1] left
[0, 0] right
Random action: [0, 1] down
[1, 1] up
[0, 1] left
[0, 0] right
Random action: [0, 1] left
[0, 0] right
[0, 1] left
[0, 0] down
[1, 0] up
[0, 0] down
[1, 0] down
[2, 0] down
[3, 0] down
[4, 0] up
[3, 0] down
Random action: [4, 0] up
[3, 0] up
[2, 0] down
[3, 0] up
[2, 0] down
Random action: [3, 0] right
[3, 1] down
[4, 1] right
Random action: [4, 2] left
[4, 1] right
[4, 2] right
Random action: [4, 3] left
Random action: [4, 2] left
[4, 1] right
[4, 2] right
[4, 3] rig

In [474]:
test(state,spielfeld,learned)

[0, 0] right
[0, 1] right
[0, 2] right
[0, 3] down
[1, 3] left
[1, 2] up
[0, 2] right
[0, 3] down
[1, 3] left
[1, 2] up
[0, 2] right
[0, 3] down
[1, 3] left
[1, 2] up
[0, 2] right
[0, 3] down
[1, 3] left
[1, 2] up
[0, 2] right
[0, 3] down
[1, 3] left
[1, 2] up
[0, 2] right
[0, 3] down


In [475]:
b = np.array([-1,0,0,-1,0,0,-1])
print(b == b.max())
np.flatnonzero(b == b.max())
            

[False  True  True False  True  True False]


array([1, 2, 4, 5], dtype=int64)

In [124]:
print(np.max([learned[23, :], learned[19, :]]))
print([learned[23, :], learned[19, :]])

23.69
[array([0., 0., 0., 0.]), array([23.69,  0.  ,  0.  ,  0.  ])]


In [241]:
learned

array([[-4.99999999e+01,  0.00000000e+00,  0.00000000e+00,
        -1.00000000e+00],
       [-1.80000000e+01,  0.00000000e+00, -9.99999743e-01,
        -1.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -9.99999972e-01,
        -9.99998084e-01],
       [-5.90400000e-01,  0.00000000e+00, -9.99009648e-01,
        -9.99867077e-01],
       [-2.53472307e+01,  0.00000000e+00, -9.64815628e-01,
         0.00000000e+00],
       [-9.99740385e-01, -9.99999933e-01,  0.00000000e+00,
        -3.68928000e+01],
       [-2.00000000e-01, -2.00000000e-01, -3.68928000e+01,
        -1.80000000e+01],
       [-2.00000000e-01, -2.00000000e-01, -1.00000000e+01,
        -2.00000000e-01],
       [-3.60000000e-01, -2.00000000e-01, -1.00000000e+01,
        -2.44000000e+01],
       [ 2.22177131e+02, -7.37856000e-01, -2.00000000e-01,
         0.00000000e+00],
       [-4.88741001e+01, -4.97048521e+01,  0.00000000e+00,
        -5.90400000e-01],
       [-1.80000000e+01, -1.00000000e+01, -2.00000000e-01,
      